In [30]:
import torch
import asyncio
from sentence_transformers import SentenceTransformer
from abc import ABC,abstractclassmethod

class BaseEmbedding(ABC):

    @abstractclassmethod
    def _generate_embeddings(self, text):
        pass


class SentenceTransformersEmbedding(BaseEmbedding):
    def __init__(self, model_name:str = "sentence-transformers/all-MiniLM-L6-v2", device:str = 'cpu',show_progress:str = False):
        super().__init__()
        self.show_progress = show_progress
        self.model_name = model_name
        self.device = device
        self.model = SentenceTransformer(self.model_name,device=self.device)

    def _generate_embeddings(self, text):
        embeddings = self.model.encode(text,show_progress_bar=self.show_progress).tolist()
        return embeddings
        
    def embed(self, text):
        return self._generate_embeddings(text)

        

In [32]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from chromadb import Documents, EmbeddingFunction, Embeddings

class ChromaDB:
    def __init__(self,embedding_model = None):
        if embedding_model == None:
            self.embedder = SentenceTransformersEmbedding(device = "cpu")
        else:
            self.embedder = embedding_model

    def save(self,documents, db_path,collection_name:str = "quickstart" ):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.create_collection(collection_name)
        embeddings = self._get_document_embeddings(documents)
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")
        
    def _get_document_embeddings(self,text):
        return self.embedder.embed(text)
        
    def retrive(self,db_path,user_input,collection_name = "quickstart"):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection(collection_name)
        # return collection  
        results = collection.query(
        query_embeddings = [self._get_document_embeddings(user_input)],
        n_results=2,
        )
        return results


In [35]:
from semantic_text_splitter import HuggingFaceTextSplitter
def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks

In [36]:
with open("grand_cholan.txt") as f:
    text = f.read()

In [ ]:
from pydantic import List
class Ensemble:

    def __init__(self,vector_db,model_names:List[str]):
        self.vector_db = vector_db
        self.model_names = model_names

    def generate_batch():
        for i in range(1,6):
            yield i
        
    def save(self,):
        # Initialize models and store in a list
        models = [CustomEmbeddingModel(model_name=name) for name in model_names]
        embeddings = model.encode_text(segmented_text)
        database_path = f"./embedded_v2/path2_{no}"
        model.save_vectors_to_chromadb(embeddings, segmented_text, database_path)
            